# HAR CNN training 

In [1]:
# Imports
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare data

In [5]:
X_train, labels_train, list_ch_train = read_data(data_path="./data/", split="train") # train
X_test, labels_test, list_ch_test = read_data(data_path="./data/", split="train") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

In [6]:
X_train, X_test = standardize(X_train, X_test)

Train/Validation Split

In [7]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train, random_state = 123)

One-hot encoding:

In [8]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

In [9]:
# Imports
import tensorflow as tf

### Hyperparameters

In [10]:
batch_size = 600       # Batch size
seq_len = 128          # Number of steps
learning_rate = 0.001
epochs = 500

n_classes = 6
n_channels = 9

### Construct the graph
Placeholders

In [11]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Build Convolutional Layers

Note: Should we use a different activation? Like tf.nn.tanh?

In [12]:
with graph.as_default():
    # (batch, 128, 9) --> (batch, 32, 18)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=4, strides=4, padding='same')
    
    # (batch, 32, 18) --> (batch, 8, 36)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=36, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=4, strides=4, padding='same')
    
    # (batch, 8, 36) --> (batch, 2, 72)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=72, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=4, strides=4, padding='same')

Now, flatten and pass to the classifier

In [13]:
with graph.as_default():
    # Flatten and add dropout
    flat = tf.reshape(max_pool_3, (-1, 2*72))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [14]:
if (os.path.exists('checkpoints-cnn') == False):
    !mkdir checkpoints-cnn

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
                    # Loss
                    loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/har.ckpt")

Epoch: 0/500 Iteration: 5 Train loss: 1.924933 Train acc: 0.168333
Epoch: 1/500 Iteration: 10 Train loss: 1.904576 Train acc: 0.160000
Epoch: 1/500 Iteration: 10 Validation loss: 1.763463 Validation acc: 0.210556
Epoch: 1/500 Iteration: 15 Train loss: 1.768057 Train acc: 0.226667
Epoch: 2/500 Iteration: 20 Train loss: 1.786669 Train acc: 0.200000
Epoch: 2/500 Iteration: 20 Validation loss: 1.716566 Validation acc: 0.242222
Epoch: 2/500 Iteration: 25 Train loss: 1.733695 Train acc: 0.245000
Epoch: 3/500 Iteration: 30 Train loss: 1.712036 Train acc: 0.268333
Epoch: 3/500 Iteration: 30 Validation loss: 1.660139 Validation acc: 0.425000
Epoch: 3/500 Iteration: 35 Train loss: 1.648885 Train acc: 0.360000
Epoch: 4/500 Iteration: 40 Train loss: 1.630492 Train acc: 0.358333
Epoch: 4/500 Iteration: 40 Validation loss: 1.582805 Validation acc: 0.570000
Epoch: 4/500 Iteration: 45 Train loss: 1.573836 Train acc: 0.413333
Epoch: 5/500 Iteration: 50 Train loss: 1.541777 Train acc: 0.443333
Epoch: 5/

Epoch: 42/500 Iteration: 380 Validation loss: 0.296501 Validation acc: 0.885556
Epoch: 42/500 Iteration: 385 Train loss: 0.324394 Train acc: 0.875000
Epoch: 43/500 Iteration: 390 Train loss: 0.354407 Train acc: 0.855000
Epoch: 43/500 Iteration: 390 Validation loss: 0.295486 Validation acc: 0.883333
Epoch: 43/500 Iteration: 395 Train loss: 0.340057 Train acc: 0.866667
Epoch: 44/500 Iteration: 400 Train loss: 0.346388 Train acc: 0.861667
Epoch: 44/500 Iteration: 400 Validation loss: 0.292819 Validation acc: 0.883889
Epoch: 44/500 Iteration: 405 Train loss: 0.342678 Train acc: 0.876667
Epoch: 45/500 Iteration: 410 Train loss: 0.288564 Train acc: 0.881667
Epoch: 45/500 Iteration: 410 Validation loss: 0.290339 Validation acc: 0.886111
Epoch: 46/500 Iteration: 415 Train loss: 0.286026 Train acc: 0.893333
Epoch: 46/500 Iteration: 420 Train loss: 0.362614 Train acc: 0.868333
Epoch: 46/500 Iteration: 420 Validation loss: 0.285090 Validation acc: 0.886667
Epoch: 47/500 Iteration: 425 Train loss:

Epoch: 83/500 Iteration: 755 Train loss: 0.230481 Train acc: 0.906667
Epoch: 84/500 Iteration: 760 Train loss: 0.204169 Train acc: 0.920000
Epoch: 84/500 Iteration: 760 Validation loss: 0.218632 Validation acc: 0.912778
Epoch: 84/500 Iteration: 765 Train loss: 0.226559 Train acc: 0.916667
Epoch: 85/500 Iteration: 770 Train loss: 0.235547 Train acc: 0.895000
Epoch: 85/500 Iteration: 770 Validation loss: 0.213097 Validation acc: 0.918333
Epoch: 86/500 Iteration: 775 Train loss: 0.177692 Train acc: 0.938333
Epoch: 86/500 Iteration: 780 Train loss: 0.215894 Train acc: 0.915000
Epoch: 86/500 Iteration: 780 Validation loss: 0.208042 Validation acc: 0.917222
Epoch: 87/500 Iteration: 785 Train loss: 0.168814 Train acc: 0.931667
Epoch: 87/500 Iteration: 790 Train loss: 0.194353 Train acc: 0.930000
Epoch: 87/500 Iteration: 790 Validation loss: 0.209566 Validation acc: 0.912222
Epoch: 88/500 Iteration: 795 Train loss: 0.208733 Train acc: 0.920000
Epoch: 88/500 Iteration: 800 Train loss: 0.195683 

Epoch: 125/500 Iteration: 1130 Train loss: 0.148643 Train acc: 0.935000
Epoch: 125/500 Iteration: 1130 Validation loss: 0.180187 Validation acc: 0.928889
Epoch: 126/500 Iteration: 1135 Train loss: 0.131868 Train acc: 0.955000
Epoch: 126/500 Iteration: 1140 Train loss: 0.157562 Train acc: 0.948333
Epoch: 126/500 Iteration: 1140 Validation loss: 0.191131 Validation acc: 0.923333
Epoch: 127/500 Iteration: 1145 Train loss: 0.128606 Train acc: 0.946667
Epoch: 127/500 Iteration: 1150 Train loss: 0.133189 Train acc: 0.958333
Epoch: 127/500 Iteration: 1150 Validation loss: 0.197685 Validation acc: 0.918333
Epoch: 128/500 Iteration: 1155 Train loss: 0.126419 Train acc: 0.951667
Epoch: 128/500 Iteration: 1160 Train loss: 0.146797 Train acc: 0.945000
Epoch: 128/500 Iteration: 1160 Validation loss: 0.195279 Validation acc: 0.920000
Epoch: 129/500 Iteration: 1165 Train loss: 0.173658 Train acc: 0.930000
Epoch: 129/500 Iteration: 1170 Train loss: 0.146447 Train acc: 0.945000
Epoch: 129/500 Iteration

Epoch: 166/500 Iteration: 1495 Train loss: 0.112626 Train acc: 0.963333
Epoch: 166/500 Iteration: 1500 Train loss: 0.125439 Train acc: 0.951667
Epoch: 166/500 Iteration: 1500 Validation loss: 0.174512 Validation acc: 0.932778
Epoch: 167/500 Iteration: 1505 Train loss: 0.092211 Train acc: 0.970000
Epoch: 167/500 Iteration: 1510 Train loss: 0.102346 Train acc: 0.971667
Epoch: 167/500 Iteration: 1510 Validation loss: 0.176725 Validation acc: 0.934444
Epoch: 168/500 Iteration: 1515 Train loss: 0.091155 Train acc: 0.966667
Epoch: 168/500 Iteration: 1520 Train loss: 0.109076 Train acc: 0.960000
Epoch: 168/500 Iteration: 1520 Validation loss: 0.177327 Validation acc: 0.933333
Epoch: 169/500 Iteration: 1525 Train loss: 0.101135 Train acc: 0.956667
Epoch: 169/500 Iteration: 1530 Train loss: 0.130064 Train acc: 0.951667
Epoch: 169/500 Iteration: 1530 Validation loss: 0.184348 Validation acc: 0.930556
Epoch: 170/500 Iteration: 1535 Train loss: 0.112489 Train acc: 0.965000
Epoch: 171/500 Iteration

Epoch: 206/500 Iteration: 1860 Train loss: 0.095991 Train acc: 0.965000
Epoch: 206/500 Iteration: 1860 Validation loss: 0.177861 Validation acc: 0.930000
Epoch: 207/500 Iteration: 1865 Train loss: 0.076417 Train acc: 0.966667
Epoch: 207/500 Iteration: 1870 Train loss: 0.095624 Train acc: 0.960000
Epoch: 207/500 Iteration: 1870 Validation loss: 0.185812 Validation acc: 0.930000
Epoch: 208/500 Iteration: 1875 Train loss: 0.073343 Train acc: 0.975000
Epoch: 208/500 Iteration: 1880 Train loss: 0.075596 Train acc: 0.980000
Epoch: 208/500 Iteration: 1880 Validation loss: 0.175616 Validation acc: 0.932778
Epoch: 209/500 Iteration: 1885 Train loss: 0.091983 Train acc: 0.963333
Epoch: 209/500 Iteration: 1890 Train loss: 0.075289 Train acc: 0.976667
Epoch: 209/500 Iteration: 1890 Validation loss: 0.176161 Validation acc: 0.933889
Epoch: 210/500 Iteration: 1895 Train loss: 0.087161 Train acc: 0.963333
Epoch: 211/500 Iteration: 1900 Train loss: 0.081336 Train acc: 0.970000
Epoch: 211/500 Iteration

Epoch: 247/500 Iteration: 2225 Train loss: 0.062701 Train acc: 0.985000
Epoch: 247/500 Iteration: 2230 Train loss: 0.060165 Train acc: 0.983333
Epoch: 247/500 Iteration: 2230 Validation loss: 0.178989 Validation acc: 0.934444
Epoch: 248/500 Iteration: 2235 Train loss: 0.069024 Train acc: 0.965000
Epoch: 248/500 Iteration: 2240 Train loss: 0.064463 Train acc: 0.970000
Epoch: 248/500 Iteration: 2240 Validation loss: 0.176882 Validation acc: 0.933889
Epoch: 249/500 Iteration: 2245 Train loss: 0.060284 Train acc: 0.978333
Epoch: 249/500 Iteration: 2250 Train loss: 0.076406 Train acc: 0.975000
Epoch: 249/500 Iteration: 2250 Validation loss: 0.180930 Validation acc: 0.933333
Epoch: 250/500 Iteration: 2255 Train loss: 0.092135 Train acc: 0.965000
Epoch: 251/500 Iteration: 2260 Train loss: 0.068684 Train acc: 0.976667
Epoch: 251/500 Iteration: 2260 Validation loss: 0.180089 Validation acc: 0.936111


In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1}
        
        batch_acc = sess.run(accuracy, feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))